In [9]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import optuna
import pandas as pd

In [3]:
input_path = 'C:\\Users\\Jesus\\Documents\\CEIA\\Aprendizaje de máquina 2\\Material de clase\\TP\\TP-AMq2\\data\\preprocessed_data.csv'


In [5]:
df=pd.read_csv(input_path)

In [18]:
# Eliminación de variables que no contribuyen a la predicción por ser muy específicas
dataset = df.drop(columns=['Item_Identifier', 'Outlet_Identifier'])

# División del dataset de train y test
df_train = dataset.loc[dataset['Set'] == 'train']
df_test = dataset.loc[dataset['Set'] == 'test']

# Eliminando columnas sin datos
df_train.drop(['Set'], axis=1, inplace=True)
df_test.drop(['Item_Outlet_Sales','Set'], axis=1, inplace=True)

# División de dataset de entrenaimento y validación
X = df_train.drop(columns='Item_Outlet_Sales') #[['Item_Weight', 'Item_MRP', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type']] # .drop(columns='Item_Outlet_Sales')


x_train, x_val, y_train,y_val = train_test_split(X, df_train['Item_Outlet_Sales'], test_size = 0.3, random_state=11)
#sampler = optuna.samplers.RandomSampler(seed=42)


def objective(trial):
    """
    Objective function for optimizing hyperparameters of a RandomForestClassifier using Optuna.
    
    Args:
        trial: A `Trial` object from Optuna that contains the state of the optimization trial.
    
    Returns:
        The accuracy score of the RandomForestClassifier using the hyperparameters suggested by Optuna.
    """
    # Definir los hiperparámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 5, 500)
    max_depth = trial.suggest_int('max_depth', 2, 500)
    min_samples_split = trial.suggest_float("min_samples_split", 0.01, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical("criterion", ['poisson','squared_error'])
    
    # Crear el clasificador con los hiperparámetros sugeridos por Optuna
    clf = RandomForestRegressor(
                                    n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    criterion=criterion
                                    )
    
    # Entrenar el clasificador y calcular la precisión en el conjunto de prueba
    clf.fit(x_train, y_train)
    #y_pred = clf.predict(X_test)
    #score = accuracy_score(y_test, y_pred)

    score = cross_val_score(clf, x_train, y_train, cv=3)
    accuracy = score.mean()

    return accuracy


C:\Users\Jesus\AppData\Local\Temp\ipykernel_12380\3423202818.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(['Set'], axis=1, inplace=True)
C:\Users\Jesus\AppData\Local\Temp\ipykernel_12380\3423202818.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(['Item_Outlet_Sales','Set'], axis=1, inplace=True)


In [19]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=50)
# Imprimir los resultados de la optimización
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

[I 2023-06-13 20:43:15,950] A new study created in memory with name: no-name-1911550e-febf-4a8d-9396-882017345fba
[I 2023-06-13 20:43:17,122] Trial 0 finished with value: -0.0009477553455904767 and parameters: {'n_estimators': 190, 'max_depth': 476, 'min_samples_split': 0.7346740023932911, 'min_samples_leaf': 6, 'criterion': 'poisson'}. Best is trial 0 with value: -0.0009477553455904767.
[I 2023-06-13 20:43:17,475] Trial 1 finished with value: 0.23412965151285625 and parameters: {'n_estimators': 33, 'max_depth': 434, 'min_samples_split': 0.6051038616257767, 'min_samples_leaf': 8, 'criterion': 'squared_error'}. Best is trial 1 with value: 0.23412965151285625.
[I 2023-06-13 20:43:23,435] Trial 2 finished with value: 0.5432521526795651 and parameters: {'n_estimators': 417, 'max_depth': 107, 'min_samples_split': 0.19000671753502962, 'min_samples_leaf': 2, 'criterion': 'squared_error'}. Best is trial 2 with value: 0.5432521526795651.
[I 2023-06-13 20:43:25,327] Trial 3 finished with value: 

Best trial: score 0.5698491764137789, params {'n_estimators': 390, 'max_depth': 317, 'min_samples_split': 0.02812808727317173, 'min_samples_leaf': 9, 'criterion': 'poisson'}
